get page of symbols from search endpoint

filter out unwanted exchanges (e.g. OTC)

for each batch of symbols (say 50):

create ws connection

send set auth token message

send quote create session message

create 2 coroutines

send coroutine iterates over page of symbols, sending quote add symbols message for each one (can this be batched?)

receive coroutine waits for messages with 10s timeout

In [ ]:
import requests

import pandas as pd

from tvDatafeed import TvDatafeed

tv = TvDatafeed()

/home/craigc/personal_workspace/tvdatafeed/tvDatafeed/main.py:175: SyntaxWarning: invalid escape sequence '\['
  out = re.search('"s":\[(.+?)\}\]', raw_data).group(1)
/home/craigc/personal_workspace/tvdatafeed/tvDatafeed/main.py:181: SyntaxWarning: invalid escape sequence '\['
  xi = re.split("\[|:|,|\]", xi)
you are using nologin method, data you access may be limited


In [29]:
def get_symbols(country):

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:134.0) Gecko/20100101 Firefox/134.0",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.tradingview.com/",
        "Origin": "https://www.tradingview.com",
        "Connection": "keep-alive",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "Priority": "u=4",
    }

    symbols_remaining = 50
    start = 0
    all_symbols = []

    while symbols_remaining > 0:

        url = f'https://symbol-search.tradingview.com/symbol_search/v3/?text=&hl=1&country={country}&lang=en&search_type=stocks&start={start}&domain=production&sort_by_country={country}'

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)

        symbols_remaining = data['symbols_remaining']
        all_symbols += data['symbols']
        start = len(all_symbols)
        
        print(f"{start} symbols fetched, {symbols_remaining} remaining")

    return all_symbols

def symbols_to_df(symbols_dict):
    df = pd.DataFrame(symbols_dict)
    df.drop(columns=['currency-logoid', 'logoid', 'provider_id', 'source2', 'source_id', 'typespecs', 'prefix'], errors='ignore', inplace=True)
    return df

In [30]:
us_symbols = get_symbols("US")

display(us_symbols[0])

us_df = symbols_to_df(us_symbols)
display(us_df.head())

50 symbols fetched, 9950 remaining
100 symbols fetched, 9900 remaining
150 symbols fetched, 9850 remaining
200 symbols fetched, 9800 remaining
250 symbols fetched, 9750 remaining
300 symbols fetched, 9700 remaining
350 symbols fetched, 9650 remaining
400 symbols fetched, 9600 remaining
450 symbols fetched, 9550 remaining
500 symbols fetched, 9500 remaining
550 symbols fetched, 9450 remaining
600 symbols fetched, 9400 remaining
650 symbols fetched, 9350 remaining
700 symbols fetched, 9300 remaining
750 symbols fetched, 9250 remaining
800 symbols fetched, 9200 remaining
850 symbols fetched, 9150 remaining
900 symbols fetched, 9100 remaining
950 symbols fetched, 9050 remaining
1000 symbols fetched, 9000 remaining
1050 symbols fetched, 8950 remaining
1100 symbols fetched, 8900 remaining
1150 symbols fetched, 8850 remaining
1200 symbols fetched, 8800 remaining
1250 symbols fetched, 8750 remaining
1300 symbols fetched, 8700 remaining
1350 symbols fetched, 8650 remaining
1400 symbols fetched,

{'symbol': 'TSLA',
 'description': 'Tesla, Inc.',
 'type': 'stock',
 'exchange': 'NASDAQ',
 'currency_code': 'USD',
 'currency-logoid': 'country/US',
 'logoid': 'tesla',
 'provider_id': 'ice',
 'source2': {'id': 'NASDAQ',
  'name': 'Nasdaq Stock Market',
  'description': 'Nasdaq Stock Market'},
 'source_id': 'NASDAQ',
 'country': 'US',
 'is_primary_listing': True,
 'typespecs': ['common']}

,symbol,description,type,exchange,currency_code,country,is_primary_listing
0,TSLA,"Tesla, Inc.",stock,NASDAQ,USD,US,True
1,NVDA,NVIDIA Corporation,stock,NASDAQ,USD,US,True
2,AAPL,Apple Inc.,stock,NASDAQ,USD,US,True
3,META,"Meta Platforms, Inc.",stock,NASDAQ,USD,US,True
4,AMZN,"Amazon.com, Inc.",stock,NASDAQ,USD,US,True


In [ ]:
def write_raw_data(raw_data, symbol):
    # print(symbol)
    with open(f"{symbol}.json", 'w') as file:
        file.write(raw_data)

def raw_data_to_json(raw_data):
    results = raw_data.split("\n")
    sections = []
    for result in results:
        parsed = tv.parse_m_format(result)
        sections.append(parsed)
    
    return sections

def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        elif isinstance(item, dict):
            if "v" in item:
                flattened.append(item["v"])
            if "p" in item:
                if "v" in item["p"][1]:
                    flattened.append(item["p"][1]["v"])
        else:
            flattened.append(item)
    return flattened

def receive_data2(self, sentinel):
    raw_data = ""
    pkt_count = 0
    while True:
        try:
            result = self.ws.recv()
            write_raw_data(result, f"receive_data2_{pkt_count}")
            pkt_count += 1
            raw_data = raw_data + result + "\n"
        except Exception as e:
            print(f"Error: {e}")
            break

        if sentinel in result:
            break

    return raw_data

TvDatafeed.receive_data = receive_data2

def get_financial_data(
    symbol: str,
    exchange: str,
) -> dict:
    """get financial ratios

    Args:
        symbol (str): symbol name
        exchange (str, optional): exchange, not required if symbol is in format EXCHANGE:SYMBOL. Defaults to None.

    Returns:
        dict
    """
    symbol = tv.format_symbol(symbol=symbol, exchange=exchange)

    tv.create_connection()

    tv.send_set_auth_token()
    tv.send_quote_create_session_msg()
    tv.send_message(
        "quote_add_symbols",
        [tv.session, symbol],
    )
    raw_data = tv.receive_data(sentinel="quote_completed")

    return raw_data

def format_float(value, decimal_places=2, default="N/A"):
    """
    Formats a float value to the specified number of decimal places.
    Returns the default string if the value is None.
    
    :param value: The float value to format
    :param decimal_places: The number of decimal places to display
    :param default: The default string to return if value is None
    :return: A formatted string or the default string
    """
    if value is None:
        return default
    format_spec = f".{decimal_places}f"
    return f"{value:{format_spec}}"

financials_cols = [
    "symbol",
    "share_price",
    "currency",
    "shares_outstanding",
    "average_volume",
    "market_cap",
    "country_code",
    "isin",
    "exchange",
    "figi_exchange",
    "figi_composite",
    "beta_1yr",
    "description",
    "price_earnings_ratio",
    "price_sales_ratio",
    "price_cash_flow_ratio",
    "price_book_ratio",
    "ev",
    "ev_ebitda",
    "return_on_assets",
    "return_on_equity",
    "return_on_invested_capital",
    "gross_margin",
    "operating_margin",
    "ebitda_margin",
    "net_margin",
    "quick_ratio",
    "current_ratio",
    "inventory_turnover",
    "asset_turnover",
    "debt_assets_ratio",
    "debt_equity_ratio",
    "long_term_debt_assets_ratio",
    "long_term_debt_equity_ratio",
]

def financials_dict_to_df(financials_dict):

    symbol = financials_dict.get('symbol')
    share_price = financials_dict.get('lp', None)
    currency = financials_dict.get('currency_code', None)
    shares_outstanding = financials_dict.get('total_shares_outstanding_current', None)
    average_volume = financials_dict.get('average_volume', None)
    market_cap = financials_dict.get('market_cap_calc', None)
    country_code = financials_dict.get('country_code', None)
    isin = financials_dict.get('isin', None)
    exchange = financials_dict.get('exchange', None)
    figi = financials_dict.get('figi', None)
    figi_exchange = figi.get("exchange-level", None) if figi else None
    figi_composite = figi.get("country-composite", None) if figi else None
    beta_1yr = financials_dict.get('beta_1_year', None)
    description = financials_dict.get('description', None)

    # Valuation ratios
    price_earnings_ratio = financials_dict.get("price_earnings", None)
    price_sales_ratio = financials_dict.get("price_sales_current", None)
    price_cash_flow_ratio = financials_dict.get("price_cash_flow_current", None)
    price_book_ratio = financials_dict.get("price_book_current", None)
    ev = financials_dict.get("enterprise_value_current", None)
    ev_ebitda = financials_dict.get("enterprise_value_ebitda_current", None)
    currency = financials_dict.get("currency", None)

    # Profitability ratios
    return_on_assets = financials_dict.get("return_on_assets_current", None)
    return_on_equity = financials_dict.get("return_on_equity_current", None)
    return_on_invested_capital = financials_dict.get("return_on_invested_capital_current", None)
    gross_margin = financials_dict.get("gross_margin_current", None)
    operating_margin = financials_dict.get("operating_margin_current", None)
    ebitda_margin = financials_dict.get("ebitda_margin_current", None)
    net_margin = financials_dict.get("net_margin_current", None)

    # Liquidity ratios
    quick_ratio = financials_dict.get("quick_ratio", None)
    current_ratio = financials_dict.get("current_ratio", None)
    inventory_turnover = financials_dict.get("invent_turnover_current", None)
    asset_turnover = financials_dict.get("asset_turnover_current", None)

    # Solvency ratios
    debt_assets_ratio = financials_dict.get("debt_to_asset_current", None)
    debt_equity_ratio = financials_dict.get("debt_to_equity_current", None)
    long_term_debt_assets_ratio = financials_dict.get("long_term_debt_to_assets_current", None)
    long_term_debt_equity_ratio = financials_dict.get("long_term_debt_to_equity_current", None)

    d = {
        "symbol": symbol,
        "share_price": share_price,
        "currency": currency,
        "shares_outstanding": shares_outstanding,
        "average_volume": average_volume,
        "market_cap": market_cap,
        "country_code": country_code,
        "isin": isin,
        "exchange": exchange,
        "figi_exchange": figi_exchange,
        "figi_composite": figi_composite,
        "beta_1yr": beta_1yr,
        "description": description,
        "price_earnings_ratio": price_earnings_ratio,
        "price_sales_ratio": price_sales_ratio,
        "price_cash_flow_ratio": price_cash_flow_ratio,
        "price_book_ratio": price_book_ratio,
        "ev": ev,
        "ev_ebitda": ev_ebitda,
        "return_on_assets": return_on_assets,
        "return_on_equity": return_on_equity,
        "return_on_invested_capital": return_on_invested_capital,
        "gross_margin": gross_margin,
        "operating_margin": operating_margin,
        "ebitda_margin": ebitda_margin,
        "net_margin": net_margin,
        "quick_ratio": quick_ratio,
        "current_ratio": current_ratio,
        "inventory_turnover": inventory_turnover,
        "asset_turnover": asset_turnover,
        "debt_assets_ratio": debt_assets_ratio,
        "debt_equity_ratio": debt_equity_ratio,
        "long_term_debt_assets_ratio": long_term_debt_assets_ratio,
        "long_term_debt_equity_ratio": long_term_debt_equity_ratio,
    }

    return pd.DataFrame([d])    

In [33]:
financials_df = pd.DataFrame(columns=financials_cols)

for idx, row in us_df.iterrows():

    print(f"{idx}/{len(us_df)} {row.symbol}")

    financial_data_raw = get_financial_data(row.symbol, row.exchange)

    sections = raw_data_to_json(financial_data_raw)
    # write_raw_data(json.dumps(sections, indent=4), f"{row.symbol}_sections")

    flattened_sections = flatten_list(sections)

    # write_raw_data(json.dumps(flattened_sections, indent=4), f"{row.symbol}_flattened_sections")

    flattened_dict = {key: value for d in flattened_sections for key, value in d.items()}
    # write_raw_data(json.dumps(flattened_dict, indent=4), f"{row.symbol}_flattened_dict")

    df = financials_dict_to_df(flattened_dict)
    # display(df)
    # break
    financials_df = pd.concat([financials_df, df], ignore_index=True)


# financials_df = financials_df.dropna(subset=['ev_ebitda', 'return_on_invested_capital'])
# financials_df['earnings_yield'] = 1 / financials_df['ev_ebitda']
# financials_df["am_rank"] = financials_df["ev_ebitda"].rank(ascending=False).astype(int)
# financials_df["ey_rank"] = financials_df["earnings_yield"].rank(ascending=False).astype(int)
# financials_df["roc_rank"] = financials_df["return_on_invested_capital"].rank(ascending=False).astype(int)
# financials_df["magic_formula_score"] = financials_df["ey_rank"] + financials_df["roc_rank"]
# financials_df_mf = financials_df.sort_values(by="magic_formula_score")
# financials_df_am = financials_df.sort_values(by="am_rank")
# display(financials_df_mf.head())
# display(financials_df_am.head())

0/10000 TSLA
receive_data2_0
receive_data2_1
1/10000 NVDA


/tmp/ipykernel_16270/1976882834.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  financials_df = pd.concat([financials_df, df], ignore_index=True)


receive_data2_0
receive_data2_1
2/10000 AAPL
receive_data2_0
receive_data2_1
3/10000 META
receive_data2_0
receive_data2_1
4/10000 AMZN
receive_data2_0
receive_data2_1
5/10000 PLTR
receive_data2_0
receive_data2_1
6/10000 AMD
receive_data2_0
receive_data2_1
7/10000 MSFT
receive_data2_0
receive_data2_1
8/10000 MSTR
receive_data2_0
receive_data2_1
9/10000 COIN
receive_data2_0
receive_data2_1
10/10000 GOOGL
receive_data2_0
receive_data2_1
11/10000 SMCI
receive_data2_0
receive_data2_1
12/10000 NFLX
receive_data2_0
receive_data2_1
13/10000 BABA
receive_data2_0
receive_data2_1
receive_data2_2
14/10000 INTC
receive_data2_0
receive_data2_1
15/10000 GOOG
receive_data2_0
receive_data2_1
16/10000 AVGO
receive_data2_0
receive_data2_1
17/10000 HOOD
receive_data2_0
receive_data2_1
18/10000 SOFI
receive_data2_0
receive_data2_1
19/10000 GME
receive_data2_0
receive_data2_1
20/10000 MARA
receive_data2_0
receive_data2_1
21/10000 SOUN
receive_data2_0
receive_data2_1
22/10000 TEM
receive_data2_0
receive_data

In [36]:
financials_df = financials_df.dropna(subset=['ev_ebitda', 'return_on_invested_capital'])
financials_df['earnings_yield'] = 1 / financials_df['ev_ebitda']
financials_df["am_rank"] = financials_df["ev_ebitda"].rank(ascending=False).astype(int)
financials_df["ey_rank"] = financials_df["earnings_yield"].rank(ascending=False).astype(int)
financials_df["roc_rank"] = financials_df["return_on_invested_capital"].rank(ascending=False).astype(int)
financials_df["magic_formula_score"] = financials_df["ey_rank"] + financials_df["roc_rank"]
financials_df_mf = financials_df.sort_values(by="magic_formula_score")
financials_df_am = financials_df.sort_values(by="am_rank")

display(financials_df_mf.head())
display(financials_df_am.head())

financials_df_mf.to_csv("us_mf_250216.csv", index=False)

,symbol,share_price,currency,shares_outstanding,average_volume,market_cap,country_code,isin,exchange,figi_exchange,...,asset_turnover,debt_assets_ratio,debt_equity_ratio,long_term_debt_assets_ratio,long_term_debt_equity_ratio,earnings_yield,am_rank,ey_rank,roc_rank,magic_formula_score
5165,NSAV,0.0004,USD,5921750000,9562341.3,2368698.0,US,US64112L2034,OTC,BBG000J40J17,...,1.999108,0.000247,0.000250,0.000246,0.000249,2234.016562,4097,1,5,6
5744,STG,6.5101,USD,5250820,1907.5,88327225.0,US,US86740P2074,Cboe One,BBG00K62SK54,...,0.952612,0.119544,0.471599,0.097720,0.385502,4.711301,4094,4,33,37
1309,CYBL,0.0052,USD,5945750000,17486436.8,30917875.0,US,US23247M2052,OTC,BBG000PV62D8,...,1.613744,0.341053,1.927937,0.066596,0.376463,0.664464,4055,43,2,45
6272,FRMO,9.7300,None,44022800,9694.0,428341643.0,US,US30262F2056,OTC,BBG000C42MR7,...,0.016365,0.000731,0.001596,0.000704,0.001536,2.420345,4091,7,41,48
6679,IDEA,0.0179,USD,39787000,26716.7,712187.0,US,US46123M1062,OTC,BBG000BP4426,...,0.885752,0.062091,0.073166,0.000000,0.000000,0.934636,4071,27,25,52


,symbol,share_price,currency,shares_outstanding,average_volume,market_cap,country_code,isin,exchange,figi_exchange,...,asset_turnover,debt_assets_ratio,debt_equity_ratio,long_term_debt_assets_ratio,long_term_debt_equity_ratio,earnings_yield,am_rank,ey_rank,roc_rank,magic_formula_score
6383,HKHC,38.00,USD,18635300,536.0,7.081422e+08,US,US4399131048,OTC,BBG000GYK8N2,...,0.094685,0.021914,0.030150,0.021914,0.030150,0.000034,1,4097,4040,8137
7483,SDRMF,3.72,AUD,279698700,4741.0,1.040479e+09,US,AU0000183543,OTC,BBG014MTTJR0,...,1.527235,0.092204,0.174231,0.046583,0.088024,0.000189,2,4096,4047,8143
398,VRTX,459.00,USD,256790000.0,1478429.3,1.178666e+11,US,US92532F1003,Cboe One,BBG000C1S2X2,...,0.475753,0.075767,0.106614,0.071769,0.100990,0.000213,3,4095,3558,7653
2183,DVAX,12.82,USD,131455000.0,1581135.2,1.685247e+09,US,US2681582019,Cboe One,BBG000BFX694,...,0.256335,0.239730,0.373614,0.235565,0.367124,0.000422,4,4094,2890,6984
977,WRBY,25.69,USD,101662180,1060687.6,3.091456e+09,US,US93403J1060,Cboe One,BBG005DWN8K8,...,1.224414,0.307484,0.582545,0.266625,0.505134,0.000539,5,4093,3754,7847
